# Lecture 9: linear model and model diagnostics 

## Instructor： 胡传鹏（博士）[Dr. Hu Chuan-Peng]

### 南京师范大学心理学院[School of Psychology, Nanjing Normal University]


##  Recap: PyMC3 &  A Simple Bayesian Linear Regression Model

* Probabilisitic Programming Language & PyMC3
* A simple Bayesian linear regression model

假设我们将探究大学生的社交网站使用和创新的自我效能感对个体创新行为的关系。

`SNS_t`:  社交网站使用强度；
`CSES_t`: 创新自我效能感；
`EIB_t`: 创新行为

数据来源：郑元瑞, 谢嘉敏, 李鹏. (2022) 社交网站使用强度对大学生创新行为的影响：一个有调节的中介模型. 心理技术与应用, 10(8), 483-491.

In [1]:
%matplotlib inline
import numpy as np 
from scipy import stats
import matplotlib.pyplot as plt
import pandas as pd
import arviz as az
import pymc3 as pm
# mpl_toolkits.mplot3d是用于三维画图的，Axes3D用于画三维图
from mpl_toolkits.mplot3d import Axes3D

WARNING (theano.link.c.cmodule): install mkl with `conda install mkl-service`: No module named 'mkl'


In [3]:
np.random.seed(123)  # 随机数种子，确保随后生成的随机数相同
data = pd.read_csv("/home/mw/input/data9464/clean.csv") # 读取数据，该数据需要提前挂载
data['SNS_t'] = (data['SNS_t'] - data['SNS_t'].mean()) / data['SNS_t'].std()     # 将变量进行标准化
data['CSES_t'] = (data['CSES_t'] - data['CSES_t'].mean()) / data['CSES_t'].std() # 将变量进行标准化
data['EIB_t'] = (data['EIB_t'] - data['EIB_t'].mean()) / data['EIB_t'].std()     # 将变量进行标准化

In [4]:
#创建一个画布fig1，该画布有两幅图ax1,ax2，画布尺寸为（10，4）
fig1, ax1 = plt.subplots(1,2,figsize=(10,4))

#在第一张图的第一个panel上画散点图
ax1[0].scatter(data['SNS_t'],data['EIB_t'] )
ax1[0].set_xlabel('SNS_t')
ax1[0].set_ylabel('EIB_t')

#在第一张图的第二个panel上画散点图
ax1[1].scatter(data['CSES_t'],data['EIB_t'] )
ax1[1].set_xlabel('CSES_t')
ax1[1].set_ylabel('EIB_t')

#在第二张图（三维空间）画散点图
fig2 = plt.figure(2)
ax2 = Axes3D(fig2)
ax2.scatter(data['SNS_t'],data['CSES_t'],data['EIB_t'] )
ax2.set_xlabel('SNS_t')
ax2.set_ylabel('CSES_t')
ax2.set_zlabel('EIB_t')
plt.show()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:16: MatplotlibDeprecationWarning: Axes3D(fig) adding itself to the figure is deprecated since 3.4. Pass the keyword argument auto_add_to_figure=False and use fig.add_axes(ax) to suppress this warning. The default value of auto_add_to_figure will change to False in mpl3.5 and True values will no longer work in 3.6.  This is consistent with other Axes classes.
  app.launch_new_instance()


<Figure size 720x288 with 2 Axes>

<Figure size 432x288 with 1 Axes>

在PyMC3中，一个简单的线性模型：

1. 通过建立线性模型的概率表达形式来构建模型。

2. 通过PyMC3对后验进行采样

3. 通过Arviz对结果进行展示，辅助统计推断

线性模型的一般形式：

$y_i = a + b_1 x_1 + b_2 x_2 + \epsilon$

$y_i = \hat{y} + \epsilon$

线性模型可以用概率的形式进行表达

$y \sim Normal(\mu_i, sigma)$      ->      y $\sim$ Normal(mu,sigma)

$\mu_i  = \alpha+\beta_1 * x1 +\beta_2 * x_2$      ->      mu = alpha + $beta_1$  * $x_1$  + $beta_2$   * $x_2$

$\alpha \sim Normal(0,1)$      ->      a $\sim$ Normal(mu,sigma)

$\beta_{i}  \sim Normal(0,1)$      ->      $b_i$ $\sim$ Normal(mu,sigma)

$\sigma \sim HalfNormal(1)$      ->      sigma $\sim$ HalfNormal(1)





#### 模型构建

In [5]:
# 在pymc3中，pm.Model()定义了一个新的模型对象，这个对象是模型中随机变量的容器
# 在python中，容器是一种数据结构，是用来管理特殊数据的对象
# with语句定义了一个上下文管理器，以 linear_model 作为上下文，在这个上下文中定义的变量都被添加到这个模型
with pm.Model() as linear_model:
    # 先验分布: alpha, beta, sigma这三个参数是随机变量
    alpha = pm.Normal('alpha',mu=0,sd=1)
    beta = pm.Normal('beta',mu=0,sd=1, shape=2)  # shape 用来表示同样的分布的参数的数量
    sigma = pm.HalfNormal('sigma',sd=1)

    # x1,x2为自变量，是之前已经载入的数据
    x1 = pm.Data("x1", data['SNS_t'])
    x2 = pm.Data("x2", data['CSES_t'])

    # 线性模型：mu是确定性随机变量，这个变量的值完全由右端值确定
    mu = pm.Deterministic("mu", alpha + beta[0]*x1 + beta[1]*x2) 

    # Y的观测值，这是一个特殊的观测随机变量，表示模型数据的可能性。也可以表示模型的似然，通过 observed 参数来告诉这个变量其值是已经被观测到了的，不会被拟合算法改变
    y_obs = pm.Normal('y_obs',mu=mu,sd=sigma,observed=data['EIB_t'] )


In [6]:
pm.model_to_graphviz(linear_model)

In [7]:
#采样过程仍在该容器中进行
with linear_model:
    # 使用mcmc方法进行采样，draws为采样次数，tune为调整采样策略的次数，这些次数将在采样结束后被丢弃，
    # target_accept为接受率， return_inferencedata=True为该函数返回的对象是arviz.InnferenceData对象
    # chains为我们采样的链数，cores为我们的调用的cpu数，多个链可以在多个cpu中并行计算，我们在和鲸中调用的cpu数为2
    trace = pm.sample(draws = 2000, tune=1000, target_accept=0.9,chains=2, cores= 2,return_inferencedata=True)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [sigma, beta, alpha]


Sampling 2 chains for 1_000 tune and 2_000 draw iterations (2_000 + 4_000 draws total) took 6 seconds.


#### 参数的后验分布
这里的模型分析结果展示了各参数的分布(后验)情况

In [8]:
# 绘制特定参数的采样情况，选取对象为trace，选取其中'alpha','beta','sigma'三个参数
az.plot_trace(trace,var_names=['alpha','beta','sigma'])

array([[<AxesSubplot:title={'center':'alpha'}>,
        <AxesSubplot:title={'center':'alpha'}>],
       [<AxesSubplot:title={'center':'beta'}>,
        <AxesSubplot:title={'center':'beta'}>],
       [<AxesSubplot:title={'center':'sigma'}>,
        <AxesSubplot:title={'center':'sigma'}>]], dtype=object)

<Figure size 864x432 with 6 Axes>

In [9]:
# 绘制特定参数的森林图，选取对象为trace，选取其中'alpha','beta','sigma'三个参数
az.plot_forest(trace,var_names=['alpha','beta','sigma'],r_hat=True)

array([<AxesSubplot:title={'center':'94.0% HDI'}>,
       <AxesSubplot:title={'center':'r_hat'}>], dtype=object)

<Figure size 576x489.6 with 2 Axes>

In [54]:
# 参数的后验分布图，选取对象为trace，选取其中'alpha', 'beta', 'sigma'三个参数
az.plot_posterior(trace,var_names=['alpha','beta','sigma'])
# beta1和beta2的后验分布值大于0的比例，若大于95%则可认为对应的自变量对因变量有影响
print(f"beta0大于0的概率为{(trace.posterior.beta[0] > 0).mean().values}")
print(f"beta1大于0的概率为{(trace.posterior.beta[1] > 0).mean().values}")

beta0大于0的概率为0.99625
beta1大于0的概率为0.998


<Figure size 1987.2x331.2 with 4 Axes>

In [46]:
# 参数的后验分布图，选取对象为trace，选取其中'alpha','beta','sigma'三个参数
az.summary(trace,var_names=['alpha','beta','sigma'])

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alpha,-0.313,0.022,-0.354,-0.271,0.0,0.0,3305.0,3167.0,1.0
beta[0],-0.003,0.023,-0.043,0.042,0.0,0.0,3337.0,2697.0,1.0
beta[1],0.780,0.023,0.739,0.825,0.0,0.0,2875.0,2759.0,1.0
sigma,0.685,0.014,0.659,0.710,0.0,0.0,3048.0,2821.0,1.0


In [83]:
# 后验预测分布的计算仍在容器中进行
with linear_model:
    # pm.sample_posterior_predictive()利用trace.posterior的后验分布计算后验预测分布
    ppc_y = pm.sample_posterior_predictive(trace.posterior) 
#将ppc_y转化为InferenceData对象合并到trace中
az.concat(trace, az.from_pymc3(posterior_predictive=ppc_y), inplace=True)

/opt/conda/lib/python3.7/site-packages/arviz/data/io_pymc3.py:100: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  FutureWarning,


In [84]:
# 绘制后验预测分布
az.plot_ppc(trace)

<AxesSubplot:xlabel='y_obs'>

/opt/conda/lib/python3.7/site-packages/IPython/core/events.py:89: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  func(*args, **kwargs)


<Figure size 432x288 with 1 Axes>

## Part 1: Workflow

## 从理论到问题解决

## $p(\theta|data) = \frac{p(data|\theta)p(\theta)}{{p(data)}}$

贝叶斯定理只是告诉我们一个解决问题框架，在实践中，我们需要一系列流程来帮助我们解决问题。

这一系列的流程，就是贝叶斯的workflow

#### 什么是workflow

贝叶斯推理是对条件概率或者概率密度的计算。

贝叶斯的workflow除了计算模型的后验分布外还包括研究问题，选择模型，选择先验，模型拟合，采样过程评估，模型诊断，模型比较，统计推断，结果报告。

#### 为什么需要workflow？

一个完整的workflow是为了更好地解决问题。

贝叶斯推断本身就是很复杂的推断过程，这一推断过程涉及到一系列的步骤。

如果忽视这个过程中的一些环节，在推断的过程中就会出现遗漏，会对结果产生影响。

#### Workflow能带给我们什么


![Image Name](https://cdn.kesci.com/upload/image/rklz1ernq7.jpg?imageView2/0/w/960/h/960)



* 贝叶斯推断中，我们在探索未知的参数空间，从数据到最终的结论，中间需要经过多个步骤，而每一步骤都有可能会出错；
* 过去十年中贝叶斯推断的发展，研究者发现使用标准化的workflow将更好地保证我们进行高质量地探索。



#### Workflow包括什么步骤？

![Image Name](https://cdn.kesci.com/upload/image/rki1v9nkk5.png?imageView2/0/w/960/h/960)

Gelman et al. (2020) 提出了一个比较完整的workflow，展示了模型分析过程中可能出现的步骤，出现的问题和相应的调整方法。

Gelman等的workflow会涉及到对计算的检查，会通过一系列的方法来验证计算过程是否是有效的，比如模拟假数据。

基于Martin等(2021)年在《Bayesian Modeling and Computation in Python》一书中也提出其workflow。

基于以上两个workflow，我们在本课程中采用一个更加简化的workflow，具体包括如下几个步骤：

研究问题、选择模型、选择先验、模型拟合、采样过程评估、模型诊断、模型比较、统计推断、结果报告。


![Image Name](https://cdn.kesci.com/upload/image/rkm3pw954u.png?imageView2/0/w/960/h/960)


#### （1）提出研究问题(driving question)

任何一个数据分析都是为了解决一个研究问题，可能是理论的问题，可能是现实的问题，比如：
* 做出笑的表情是否会让人更高兴？
* 自我相关的信息是否会被优先处理？
* 团体辅导是否会是比较有效的干预手段？
* 广告的投放是否增加了销量
* ...

从问题中提炼出统计问题并能够收集能够检验（某种程度上）回答这些问题的数据需要的是在某个领域的专业知识。

例子：

研究问题：社交网站的使用与是否与个体的创新行为有关系？可能的影响因素是什么？

#### （2）数据收集
略


例子：

采用问卷测量大学生三种心理特质：社交网站使用、自我交通感、创新行为。

#### （3）选择模型

根据对研究问题的背景知识和数据的特点，选择合适的模型来对数据进行解释。

模型的选择并非一蹴而就，而是需要反复迭代。

Workflow的许多步骤即是为了对模型进行诊断和调整。

在贝叶斯workflow里，我们需要用概率的形式对模型进行表达。

例子：

研究问题中涉及到一个因变量和两个自变量，我们可以构建最简单的线性模型。

线性模型的一般形式：

$\hat{y} = a + b_1 x_1 + b_2 x_2$

$y_i = \hat{y} + \epsilon$

线性模型可以用概率的形式进行表达：

$\mu_i  = \alpha+\beta_1 * x1 +\beta_2 * x_2$

$y \sim Normal(\mu_i, \sigma)$     

这就是模型的似然。

模型涉及到4个需要估计的参数：$\alpha$、$\beta_1$、$\beta_2$、$\sigma$。

#### （4）选择先验 

$p(\theta|data) \propto p(data|\theta)p(\theta)$

在贝叶斯推断中，模型的似然函数与参数的先验分布是进行推断的前提。

* Lecture #5 中：先验会影响后验分布
* 先验分布的选择需要领域的相关知识，
* 先验的设定是可以调整的（例如，通过先验预测检查, prior predicitive check)

例子：

社交网站使用、自我效能感与创新能力的简单回归模型涉及到四个参数。

由于所有的数据均已经进行标准化（$Z$值），所以我们可以合理地认为其回归模型的参数也服从标准正态：

$\alpha \sim Normal(0,1)$    

$\beta_{i}  \sim Normal(0,1)$  

$\sigma \sim HalfNormal(1)$  


##### （4.1）先验预测检验(Prior Predictive Check)

先验的设定是否合理？

可以通过先验预测检验（ Prior Predictive Distribution ）来进行初步的判断。

先验预测检验：利用模型和先验生成假数据并利用这些生成的假数据来评估先验是否合理的过程。

In [10]:
with pm.Model() as linear_model:
    # 先验分布: alpha, beta, sigma这三个参数是随机变量
    alpha = pm.Normal('alpha',mu=0,sd=1)
    beta = pm.Normal('beta',mu=0,sd=1, shape=2)  # shape 用来表示同样的分布的参数的数量
    sigma = pm.HalfNormal('sigma',sd=1)
    
    # 先验预测检查
    prior_checks = pm.sample_prior_predictive(samples=50)

In [11]:
fig = plt.figure()
ax = Axes3D(fig) #讲画布对象改为3d对象
x1 = np.linspace(-3, 3, 50) #生成从-2，2之间的50个假数据
x2 = np.linspace(-3, 3, 50)

for a, b in zip(prior_checks["alpha"], prior_checks["beta"]):
    y = a + b[0] * x1 + b[1]*x2 # 基于假数据生成预测值
    ax.plot(x1, x2, y)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: MatplotlibDeprecationWarning: Axes3D(fig) adding itself to the figure is deprecated since 3.4. Pass the keyword argument auto_add_to_figure=False and use fig.add_axes(ax) to suppress this warning. The default value of auto_add_to_figure will change to False in mpl3.5 and True values will no longer work in 3.6.  This is consistent with other Axes classes.
  


<Figure size 432x288 with 1 Axes>

我们可以看到，基于参数的先验生成的因变量的取值范围在-4-6之间，与真实情况相差不大，可以接受。

若我们改变一下先验，则会看到因变量取值范围在-100-100之间，这与真实情况相差较大，故需要调整

In [12]:
with pm.Model() as linear_model:
    # 先验分布: alpha, beta, sigma这三个参数是随机变量
    alpha = pm.Normal('alpha', mu=0, sd=10)
    beta = pm.Normal('beta', mu=0, sd=10, shape=2)  # shape 用来表示同样的分布的参数的数量
    sigma = pm.HalfNormal('sigma', sd=1)
    
    # 先验预测检查
    prior_checks = pm.sample_prior_predictive(samples=50)

In [13]:
fig = plt.figure()
ax = Axes3D(fig) #讲画布对象改为3d对象
x1 = np.linspace(-3, 3, 50) #生成从-2，2之间的50个假数据
x2 = np.linspace(-3, 3, 50)

for a, b in zip(prior_checks["alpha"], prior_checks["beta"]):
    y = a + b[0] * x1 + b[1]*x2 #基于假数据生成预测值
    ax.plot(x1,x2,y)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: MatplotlibDeprecationWarning: Axes3D(fig) adding itself to the figure is deprecated since 3.4. Pass the keyword argument auto_add_to_figure=False and use fig.add_axes(ax) to suppress this warning. The default value of auto_add_to_figure will change to False in mpl3.5 and True values will no longer work in 3.6.  This is consistent with other Axes classes.
  


<Figure size 432x288 with 1 Axes>

#### （5）拟合数据

设置好模型及其先验后，即可对数据进行拟全，从而获得关于参数的后验。

$y \sim Normal(\mu_i, sigma)$      ->      y $\sim$ Normal(mu, sigma)

$\mu_i  = \alpha+\beta_1 * x1 +\beta_2 * x_2$      ->      mu = alpha + $beta_1$  * $x_1$  + $beta_2$ * $x_2$

$\alpha \sim Normal(0,1)$      ->      a $\sim$ Normal(mu=0, sigma=1)

$\beta_{i}  \sim Normal(0,1)$      ->      $b_i$ $\sim$ Normal(mu=0, sigma=1)

$\sigma \sim HalfNormal(1)$      ->      sigma $\sim$ HalfNormal(1)


In [14]:
#首先，载入pymc3这个包，将其命名为pm
import pymc3 as pm
# 在pymc3中，pm.Model()定义了一个新的模型对象，这个对象是模型中随机变量的容器
# 在python中，容器是一种数据结构，是用来管理特殊数据的对象
# with语句定义了一个上下文管理器，以 linear_model 作为上下文，在这个上下文中定义的变量都被添加到这个模型
with pm.Model() as linear_model:
    # 先验分布: alpha, beta, sigma这三个参数是随机变量
    alpha = pm.Normal('alpha', mu=0, sd=1)
    beta = pm.Normal('beta', mu=0,sd=1, shape=2)  # shape 用来表示同样的分布的参数的数量
    sigma = pm.HalfNormal('sigma', sd=1)

    # x1,x2为自变量，是之前已经载入的数据
    x1 = pm.Data("x1", data['SNS_t'])
    x2 = pm.Data("x2", data['CSES_t'])

    # 线性模型：mu是确定性随机变量，这个变量的值完全由右端值确定
    mu = pm.Deterministic("mu", alpha + beta[0]*x1 + beta[1]*x2) 

    # Y的观测值，这是一个特殊的观测随机变量，表示模型数据的可能性。也可以表示模型的似然，通过 observed 参数来告诉这个变量其值是已经被观测到了的，不会被拟合算法改变
    y_obs = pm.Normal('y_obs',mu=mu,sd=sigma,observed=data['EIB_t'] )

#### （6）评估/诊断MCMC

如果使用MCMC对后验进行近似，则需要首先对MCMC过程进行评估。

* 是否收敛；
* 是否接近真实的后验。

对采样过程的评估我们会采用目视检查或rhat这个指标。

In [15]:
#采样过程仍在该容器中进行
with linear_model :
    # 使用mcmc方法进行采样，draws为采样次数，tune为调整采样策略的次数，这些次数将在采样结束后被丢弃，
    # target_accept为接受率， return_inferencedata=True为该函数返回的对象是arviz.InnferenceData对象
    # chains为我们采样的链数，cores为我们的调用的cpu数，多个链可以在多个cpu中并行计算，我们在和鲸中调用的cpu数为2
    trace = pm.sample(draws = 2000, tune=1000, target_accept=0.9,chains=2, cores= 2,return_inferencedata=True)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [sigma, beta, alpha]


Sampling 2 chains for 1_000 tune and 2_000 draw iterations (2_000 + 4_000 draws total) took 5 seconds.


#### （7）模型诊断

在MCMC有效的前提下，需要继续检验模型是否能够较好地拟合数据。

我们会使用后验预测分布通过我们得到的参数生成一批模拟数据，并将其与真实数据进行对比。


In [16]:
# 后验预测分布的计算仍在容器中进行
with linear_model:
    # pm.sample_posterior_predictive()利用trace.posterior的后验分布计算后验预测分布
    ppc_y = pm.sample_posterior_predictive(trace.posterior) 
#将ppc_y转化为InferenceData对象合并到trace中
az.concat(trace, az.from_pymc3(posterior_predictive=ppc_y), inplace=True)

/opt/conda/lib/python3.7/site-packages/arviz/data/io_pymc3.py:100: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  FutureWarning,


In [17]:
# 绘制后验预测分布
az.plot_ppc(trace)

<AxesSubplot:xlabel='y_obs'>

/opt/conda/lib/python3.7/site-packages/IPython/core/events.py:89: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  func(*args, **kwargs)


<Figure size 432x288 with 1 Axes>

#### （8）模型比较

到目前为止，我们已经使用后验预测检查来独立评估已有的模型。

这种类型的评估对于单独理解每个模型很有用。然而，当我们有多个模型时，这就引出了模型相对于彼此的性能如何的问题。

模型比较可以进一步帮助我们了解一个模型在哪些区域可能表现良好。

首先，我们构建一个新的模型，假定该模型中其他条件均不变，观测值服从gumbel分布

In [18]:
with pm.Model() as linear_model_2:
    # 先验分布: alpha, beta, sigma这三个参数是随机变量
    alpha = pm.Normal('alpha',mu=0,sd=1)
    beta = pm.Normal('beta',mu=0,sd=1, shape=2)  # shape 用来表示同样的分布的参数的数量
    sigma = pm.HalfNormal('sigma',sd=1)
    #先验预测检查
    prior_checks = pm.sample_prior_predictive(samples=50)

In [19]:
# 在pymc3中，pm.Model()定义了一个新的模型对象，这个对象是模型中随机变量的容器
# 在python中，容器是一种数据结构，是用来管理特殊数据的对象
# with语句定义了一个上下文管理器，以 linear_model 作为上下文，在这个上下文中定义的变量都被添加到这个模型
with pm.Model() as linear_model_2:
    # 先验分布: alpha, beta, sigma这三个参数是随机变量
    alpha = pm.Normal('alpha',mu=0,sd=1)
    beta = pm.Normal('beta',mu=0,sd=1, shape=2)  # shape 用来表示同样的分布的参数的数量
    sigma = pm.HalfNormal('sigma',sd=1)

    # x1,x2为自变量，是之前已经载入的数据
    x1 = pm.Data("x1", data['SNS_t'])
    x2 = pm.Data("x2", data['CSES_t'])

    # 线性模型：mu是确定性随机变量，这个变量的值完全由右端值确定
    mu = pm.Deterministic("mu", alpha + beta[0]*x1 + beta[1]*x2) 

    # Y的观测值，这是一个特殊的观测随机变量，表示模型数据的可能性。也可以表示模型的似然，通过 observed 参数来告诉这个变量其值是已经被观测到了的，不会被拟合算法改变
    y_obs = pm.Normal('y_obs',mu=mu,sd=sigma,observed=data['EIB_t'] )

In [20]:
#采样过程仍在该容器中进行
with linear_model_2 :
    # 使用mcmc方法进行采样，draws为采样次数，tune为调整采样策略的次数，这些次数将在采样结束后被丢弃，
    # target_accept为接受率， return_inferencedata=True为该函数返回的对象是arviz.InnferenceData对象
    # chains为我们采样的链数，cores为我们的调用的cpu数，多个链可以在多个cpu中并行计算，我们在和鲸中调用的cpu数为2
    trace2 = pm.sample(draws = 2000, tune=1000, target_accept=0.9,chains=2, cores= 2,return_inferencedata=True)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [sigma, beta, alpha]


Sampling 2 chains for 1_000 tune and 2_000 draw iterations (2_000 + 4_000 draws total) took 5 seconds.


In [21]:
# 后验预测分布的计算仍在容器中进行
with linear_model_2:
    # pm.sample_posterior_predictive()利用trace.posterior的后验分布计算后验预测分布
    ppc_y = pm.sample_posterior_predictive(trace2.posterior) 
#将ppc_y转化为InferenceData对象合并到trace中
az.concat(trace2, az.from_pymc3(posterior_predictive=ppc_y), inplace=True)

/opt/conda/lib/python3.7/site-packages/arviz/data/io_pymc3.py:100: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  FutureWarning,


In [22]:
# 绘制后验预测分布
az.plot_ppc(trace2)

<AxesSubplot:xlabel='y_obs'>

/opt/conda/lib/python3.7/site-packages/IPython/core/events.py:89: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  func(*args, **kwargs)


<Figure size 432x288 with 1 Axes>

对于两个模型哪个更好。从我们的视觉检查来看，似乎不能直接看出区别。我们可以使用 ArviZ 中的比较方法来比较这一观察结果。

In [23]:
compare_dict = {"normal": trace,"gumbel": trace2}
comp = az.compare(compare_dict, ic="loo")
comp

/opt/conda/lib/python3.7/site-packages/arviz/stats/stats.py:146: UserWarning: The default method used to estimate the weights for each model,has changed from BB-pseudo-BMA to stacking
  "The default method used to estimate the weights for each model,"


,rank,loo,p_loo,d_loo,weight,se,dse,warning,loo_scale
normal,0,-938.657710,5.237795,0.000000,1.0,30.336169,0.000000,False,log
gumbel,1,-938.760429,5.361828,0.102719,0.0,30.342870,0.038919,False,log


In [39]:
az.plot_compare(comp)

<AxesSubplot:xlabel='Log'>

<Figure size 432x144 with 1 Axes>

我们一般会选择loo值低的模型作为最佳模型，除了loo外还有AIC。BIC等指标，我们会在以后的课程中给大家讲解。

#### （9）统计推断

在选择最佳模型之后，我们就会根据最佳模型进行统计推断。



In [55]:
# 参数的后验分布图，选取对象为trace，选取其中'alpha','beta','sigma'三个参数
az.plot_posterior(trace,var_names=['alpha','beta','sigma'])
# beta1和beta2的后验分布值大于0的比例，若大于95%则可认为对应的自变量对因变量有影响
print(f"beta0大于0的概率为{(trace.posterior.beta[0] > 0).mean().values}")
print(f"beta1大于0的概率为{(trace.posterior.beta[1] > 0).mean().values}")

beta0大于0的概率为0.99625
beta1大于0的概率为0.998


<Figure size 1987.2x331.2 with 4 Axes>

由于beta1和beta2的采样值有超过99%都大于0，于是我们可以认为这两个变量对因变量产生了影响。

#### （10）报告结果

结果的报告需要看视对象而不同：
* 专业的读者（向学术期刊投稿）
* 大领域同行（学术会议上的报告）
* 非同行的其他研究者
* 非研究者（科普）
* 决策者

# Practice


## Workflow

#### 研究问题

还有研究发现个体创新行为可能与自尊水平有关。
SES_t:自尊水平；
EIB_t创新行为

试探究两者之间的关系。

In [24]:
data['SES_t'] = (data['SES_t'] - data['SES_t'].mean()) / data['SES_t'].std()#将变量进行标准化

data['EIB_t'] = (data['EIB_t'] - data['EIB_t'].mean()) / data['EIB_t'].std()#将变量进行标准化

plt.scatter(data['SES_t'],data['EIB_t'])
plt.xlabel('SES_t')
plt.ylabel('EIB_t')

Text(0, 0.5, 'EIB_t')

<Figure size 432x288 with 1 Axes>

#### 选择模型

尝试构建一个简单的线性模型

线性模型可以用概率的形式进行表达


$\alpha \sim Normal(0,1)$      ->      a $\sim$ Normal(mu,sigma)

$\beta  \sim Normal(0,1)$      ->      b $\sim$ Normal(mu,sigma)

$\sigma \sim HalfNormal(1)$      ->      sigma $\sim$ HalfNormal(1)

$\mu_i  = \alpha + \beta *x$      ->      mu = alpha + beta*x 

$y \sim Normal(\mu_i,sigma)$      ->      y $\sim$ Normal(mu,sigma)

#### 选择先验

In [ ]:
########################################################
# 练习
# 请尝试用代码表达模型的先验
########################################################
with pm.Model() as linear_model:
    # 先验分布: alpha, beta, sigma这三个参数是随机变量
    alpha = ...
    beta = pm.Normal('beta',mu=0,sd=1)  
    sigma = ...
    
    # 先验预测检查
    prior_checks = pm.sample_prior_predictive(samples=50)

In [ ]:
fig = plt.figure()
x1 = np.linspace(-3, 3, 50) #生成从-2，2之间的50个假数据

for a, b in zip(prior_checks["alpha"], prior_checks["beta"]):
    y = a + b * x1 
    ax.plot(x1,y)

#### 模型拟合

In [ ]:
########################################################
# 练习
# 请尝试表达自变量与预测值的线性关系,并将...替换为表达式
########################################################
linear_model = pm.Model()
with linear_model :
    #
    alpha = pm.Normal('alpha',mu=0,sd=1)
    beta = pm.Normal('beta',mu=0,sd=1,shape=2)
    sigma = pm.HalfNormal('sigma',sd=1)
    #
    x = pm.Data("x", x)
    #
    mu = pm.Deterministic("mu", ...) 
    #
    y_obs = pm.Normal('y_obs',mu=mu,sd=sigma,observed=y)

#### 采样过程评估

In [ ]:
########################################################
# 练习
# 请尝试调用pm.sample()函数，用mcmc进行采样
########################################################
...

In [ ]:
########################################################
# 练习
# 请尝试调用pm.plot_trace()函数，检查模型收敛情况
########################################################
...

## Part 2: MCMC Diagnostic

在贝叶斯推断中，通过概率统计语言 PyMC3 实现 MCMC 是关键的步骤。
```python
with model:
    trace = pm.sample(draws = 2000, chains=2)
```
比如在上述代码示例中，我们对贝叶斯模型采样2000个样本，并同时运行2条马尔科夫链。

在贝叶斯的workflow中，也强调需要对MCMC进行检验，检验的核心在于： 

- 这2000个样本真的能代表目标分布吗？
- 我们如何检测MCMC的样本是否存在问题？

注意：
- PyMC 的结果 traces 中包含多个 MCMC chains
- 每个 chain 中可能包括多个参数
- 每个参数的采样都可以代表为一个参数后验分布(目标分布)

为了方便，接来的讨论的**单个chain**都只包含**单个参数**。

### 什么是 MCMC diagnostic?

为了检验MCMC 样本的质量与问题，我们需要一些诊断工具，这些诊断工具统称为：马尔可夫链蒙特卡罗诊断(Markov Chain Monte Carlo diagnostics)，简称 MCMC diagnostics。

MCMC diagnostic 可以用来检查用MCMC算法采样得到的样本是否足以提供目标分布的精确近似。

MCMC diagnostic 的目的：
1. 发现 MCMC 样本存在的问题。
2. 根据诊断对 MCMC 算法进行修正(fix)。

MCMC diagnostic 检查的具体内容:
- MCMC样本的大部分采样是否来自目标分布(target distribution)。
- MCMC样本量是否足够近似目标分布。

### MCMC 样本的两大问题

要了解 MCMC diagnostic 检测到底是什么，首先我们需要知道 MCMC 样本最常见的问题。

MCMC 样本最常见的两大问题：
1. bias: 样本是否代表整个后验分布?
2. precision: 是否有足够的样本来获得足够精确的统计推断?

这两个问题涉及到 MCMC 诊断的两个专有概念：收敛性与有效样本量。

1. 收敛(convergence)问题。
   
   由于马尔科夫链的性质，无论 MCMC 算法的起始值(initial or starting value)和起始分布为何，随着采样的进行，样本最终都应该接近目标分布。

   而由于各种问题(比如，代码错误，或者先验定义错误)，样本无法接近目标分布，我们称为 MCMC chain **无法收敛**。见下图2。
   
   即使 MCMC chain 可以收敛，如果起始值离目标分布太远，那么它收敛所需的时间也更长，见下图3。

图1：参数收敛时MCMC采样的样本


![Image Name](https://cdn.kesci.com/upload/image/rkijyv4px4.png?imageView2/0/w/300/h/640)


图2：参数不收敛时MCMC采样的样本

![Image Name](https://cdn.kesci.com/upload/image/rkijynzzao.png?imageView2/0/w/300/h/640)

图3：起始值过于远离目标分布时的MCMC采样

![Image Name](https://cdn.kesci.com/upload/image/rkijy7xre5.png?imageView2/0/w/300/h/640)

source: https://statlect.com/fundamentals-of-statistics/Markov-Chain-Monte-Carlo-diagnostics

2. 有效样本量(effective sample size)过于少的问题。

   在MCMC过程中，样本量越多，样本分布越接近目标分布；但是当样本数量过少时，可能对目标分布产生错误的推断。那什么因素会导致采样样本的数量变少？
   
   由于马尔科夫链的性质，当前采样 $\theta_{t}$ 依赖于 上一次的采样 $\theta_{t-1}$。因此，样本间的关系不是独立的，这违反了独立采样的假设。违反独立采样的后果可能就是 MCMC chain 无法收敛，如下图2。

   如果 $\theta_{t}$ 与 $\theta_{t-n}$ 的间隔 n 越大，两个样本的相关性越低。

   **通过每隔 n 保留一个采样可以减少样本相关性的影响，但代价就是有效样本量变少**。

### MCMC 视觉检查 (Visual Diagnostics)

检查 MCMC 采样质量最直观的方式就是视觉检查 (Visual Diagnostics)。

上面的采样轨迹图就是视觉检查的一种形式。所以我们对此并不陌生。

常见的视觉检查有两类：
- 轨迹图 Trace plots。即上面的图。
- 自相关函数图 Autocorrelation function (ACF) plots。
  
  前面提到样本间存在相关性，而这种自相关性会导致轨迹图“不收敛”。而不收敛的情况有很多，并不能从轨迹图中得知是因为自相关导致了不收敛，因此我们需要自相关函数图来诊断是否存在自相关的问题。

接下来我们结合代码示例进行演示。

以前使用过的 `ArviZ` 工具包就可以完成大部分的诊断工作。所以接下来的分析会依赖于  `ArviZ` 。

为了演示上述不同问题对于 MCMC 采样的影响，我们生成了三种 MCMC chain:

- good_chains 包含两条 MCMC chain，生成自一个 $\beta$ 分布，并且两次采样间是完全独立的。
- bad_chains0 在 good_chains 的基础上，加入了采样间的自相关。
- bad_chains1 在 good_chains 的基础上，在采样的局部加入高度自相关。bad_chains1 代表了一个非常常见的场景，采样器可以很好地解析参数空间的一个区域，但是很难对一个或多个区域进行采样。


> 代码源自：2.4 Diagnosing Numerical Inference in Martin, from O. A., Kumar, R., & Lao, J. (2021). Bayesian Modeling and Computation in Python (1st ed.). Chapman and Hall/CRC. https://doi.org/10.1201/9781003019169


In [70]:
good_chains = stats.beta.rvs(2, 5,size=(2, 2000))
bad_chains0 = np.random.normal(np.sort(good_chains, axis=None), 0.05, size=4000).reshape(2, -1)
bad_chains1 = good_chains.copy()

for i in np.random.randint(1900, size=4):
  bad_chains1[i%2:, i:i+100] = np.random.beta(i, 950, size=100)

chains = {"good_chains":good_chains,
          "bad_chains0":bad_chains0,
          "bad_chains1":bad_chains1}

#### Trace Plots

轨迹图(trace plot)是贝叶斯中最常见的诊断图，横坐标为每个迭代步骤，纵坐标为采样值。

从这些图中，我们能够看到 
- MCMC chain 是否收敛。
- 同一个参数每条链是否收敛为相同的分布。
- 初步观察自相关程度。
  
在ArviZ中，通过调用函数az.plot_trace(.)，可以在右侧得到轨迹图，在左侧得到参数分布图。

In [71]:
az.plot_trace(chains)
plt.show()

<Figure size 864x432 with 6 Axes>

可以看到，
- good_chains 的表现最好，并且其中的两条 chain 几乎重合。
- bad_chains0 并不收敛，其中的两条 chain 并不趋向于同一个目标分布，并且两者有持续扩大的趋势。
- bad_chains1 虽然收敛，但是采样不均匀，即多个波峰(左图)，表现为在局部过多采样(右图)。

此外，如前面提到的，对于多条 MCMC 链，trace plot 还可以观测到不同初始值对采样的影响。

![](https://bcss.org.my/tut/wp-content/uploads/2020/04/rw3.gif)

source: https://bcss.org.my/tut/bayes-with-jags-a-tutorial-for-wildlife-researchers/mcmc-and-the-guts-of-jags/mcmc-diagnostics/

比如，由于初始值过于极端，导致需要花费多次迭代才能达到平稳分布。

![Image Name](https://cdn.kesci.com/upload/image/rkijy7xre5.png?imageView2/0/w/400/h/640)

#### Autocorrelation Plots

正如我们在讨论有效样本容量时所看到的，自相关减少了样本中包含的样本量。

因此，我们希望自相关尽量小的同时样本量足够多。

可以用`az.plot_autocorr`绘制自相关图(autocorrelation plot)检查自相关性。

In [72]:
az.plot_autocorr(chains, combined=True) # combined=True表明将两条chains合并为1条
plt.show()

<Figure size 1490.4x331.2 with 3 Axes>

上图为自相关函数在100步间隔窗口上的条形图。
- 横坐标为 $\theta_{t}$ 与 $\theta_{t+n}$ 的间隔 n，该间隔越大，两个样本的相关性越低
- 纵轴为 $\theta_{t}$ 与 $\theta_{t+n}$ 的相关性。
- 灰色带表示所有相关系数形成的95%置信区间。

该图结果显示：
- good_chains 的纵向高度接近于零(并且大部分在灰色带内)，这表明自相关性非常低。
- bad_chains0 和 bad_chains1 中的高条表示自相关性值较大。
- bad_chains1 随着 n 的间隔增大，自相关减少。而 bad_chains0 不会随着 n 增大减少，说明该 MCMC chain的自相关问题非常严重。

为了更好的理解自相关系数(y轴)如何计算，以及 间隔n对自相关的影响。

我们可以自定义一个函数：
- 首先，取出 $\theta_{t}$
- 然后，取出 $\theta_{t+n}$
- 最后计算两者的相关系数

需要注意，在当前的例子中，t的总长度为2000，每当 n 增加1，$\theta_{t}$ 和 $\theta_{t+n}$ 的长度就会减少1。因为间隔不可能超过2000。

In [73]:
def autocorr(lag, chain):
    n = lag                                                 # 定义间隔n的大小
    t = 2000                                                # 单条 chain的总长度
    theta_t = chain[0:(t-1-n)]                              # 提取 theta_t
    theta_tn = chain[n:(t-1)]                               # 提取 theta_t+n
    rho = np.corrcoef(theta_t,theta_tn)                     # 计算相关系数
    print("间隔为", n, "时，的相关系数", rho.round(2)[0][1])   # 输出结果

In [74]:
# 间隔 n = 1时，三条链的自相关性
n = 1
autocorr(n, chains["good_chains"][0])
autocorr(n, chains["bad_chains0"][0])
autocorr(n, chains["bad_chains1"][0])

间隔为 1 时，的相关系数 0.02
间隔为 1 时，的相关系数 0.64
间隔为 1 时，的相关系数 0.31


In [75]:
# 间隔 n = 2时，三条链的自相关性
n = 2
autocorr(n, chains["good_chains"][0])
autocorr(n, chains["bad_chains0"][0])
autocorr(n, chains["bad_chains1"][0])

间隔为 2 时，的相关系数 -0.0
间隔为 2 时，的相关系数 0.65
间隔为 2 时，的相关系数 0.29


In [76]:
# 间隔 n = 3时，三条链的自相关性
n = 3
autocorr(n, chains["good_chains"][0])
autocorr(n, chains["bad_chains0"][0])
autocorr(n, chains["bad_chains1"][0])

间隔为 3 时，的相关系数 0.01
间隔为 3 时，的相关系数 0.64
间隔为 3 时，的相关系数 0.29


In [77]:
# 间隔 n = 50时，三条链的自相关性
n = 50
autocorr(n, chains["good_chains"][0])
autocorr(n, chains["bad_chains0"][0])
autocorr(n, chains["bad_chains1"][0])

间隔为 50 时，的相关系数 -0.02
间隔为 50 时，的相关系数 0.63
间隔为 50 时，的相关系数 0.11


可以看到，结果与 autocorrelation的图结果一致：
- good_chains 的结果始终在0附近，表明自相关低。
- bad_chains0 的结果显示，当间隔n扩大到50时，自相关系数从0.64下降到0.63。
- bad_chains1 的结果显示，当间隔n扩大到50时，自相关系数从0.15下降到0.08附近。

上述计算过程可以通过 Arviz自带的 `autocorr`函数进行计算。

In [78]:
n = [1,2,3,50]  # 定义间隔为 1，2，3 和 50
rhos = az.autocorr(chains["good_chains"][0]).round(2)[n]
print("good_chains的相关系数：",rhos)
rhos = az.autocorr(chains["bad_chains0"][0]).round(2)[n]
print("good_chains的相关系数：",rhos)
rhos = az.autocorr(chains["bad_chains1"][0]).round(2)[n]
print("good_chains的相关系数：",rhos)

good_chains的相关系数： [ 0.02 -0.    0.01 -0.02]
good_chains的相关系数： [0.64 0.64 0.63 0.58]
good_chains的相关系数： [0.3  0.29 0.29 0.11]


通过轨迹图和自相关图，我们可以诊断出 MCMC chain 中存在的不同问题。

但自然而然能想到的两个问题是：
- 通过视觉方法判断 MCMC 的问题是否主观？能否通过客观的指标进行诊断？
- 发现了  MCMC 的问题，我们如何进行修正？

### MCMC 诊断指标

通过绘图对MCMC进行检查的优势在于直观，并且能通过图推测问题在什么地方。

但对于许多参数的模型来说，挨个检查图形是非常困难的。

因此，通过数值进行判断一方面可以提高诊断的客观性，另一方面提高了诊断的效率。

针对 MCMC 的两大问题，这里存在两个主要的指标：
1. 针对收敛问题的：Rhat，$\hat{R}$，也称为“潜在规模缩减因子” (Potential Scale Reduction Factor, PSRF)。它将单个链样本的变异与混合了所有链样本的变异进行比较。
2. 针对有效样本量问题的：effective sample size (ESS) or effective number of draws (n.eff)。

#### Convergence and Rhat

前面提到，收敛(convergence)描述了 MCMC 样本两个层面的问题：
1. MCMC 采样是否从初始值“收敛”到一个稳定的分布。
2. 即使链收敛到了一个稳定的分布，多条链形成的稳定分布是否相似。

反面例子就是 bad_chains0: 

该采样既没有收敛到一个稳定的分布(右图中采样整体向上偏移)，并且两个链形成的分布也不相同(左图)。
![Image Name](https://cdn.kesci.com/upload/image/rkimd3udb3.png?imageView2/0/w/640/h/640)

Rhat的作用就是通过一个指标反应上述两个层面的问题。

具体逻辑为：
- 运行多条 MCMC 链，并且以不同的初始值开始采样。
- 比较每条链的相似性。

数学逻辑为：
- 计算θ的所有样本的标准差，包括所有链的样本
- 计算每条链标准偏差，并通过平方平均数整合到一起
- 比较两个标准差的大小。如果两个的除数为1，说明两个变异相同，表明他们存在相似性。
- 如果，该值大于1.01，说明两个变异可能存在差异，即参数可能不收敛。

source: 
- https://www.r-bloggers.com/2020/01/applied-bayesian-statistics-using-stan-and-r/

假设，参数 $\theta$ 有m个 chain 和 n个 样本。

1. 用 $s_m^2$ 表示每条链内部的方差。那么链内部的变异 within-chain variance $W$ 为所有链方差的均值 $W = \frac{1}{M} \, \sum_{m=1}^M s_m^2$
2. 用 $\bar{s}_m^2$ 表示所有链均值计算的方差。链间的变异 between-chain variance $B$ 为 $B = \bar{s}_m^2$。
3. 结合在一起得到 Rhat $\hat{R} = \sqrt{\frac{W+B}{W}}$

可见，原理和方差分析类似，如果链间变异B趋近于0，那么Rhat趋近于1，说明各链的均值几乎相同，以此反应他们的相似性。

这里的数学公式有所简化，详细请看 https://mc-stan.org/docs/reference-manual/notation-for-samples-chains-and-draws.html。

Rhat计算的条件是有多条 MCMC 链并且尽可能的每条链使用不同的起始值。

我们可以通过 `az.rhat(.)` 简化计算过程。

In [79]:
az.rhat(chains)

<xarray.Dataset>
Dimensions:      ()
Data variables:
    good_chains  float64 1.0
    bad_chains0  float64 2.406
    bad_chains1  float64 1.031

从这个结果中我们可以看出，
- good_chains 的 Rhat 接近1，表示该参数的多条链都是收敛并且相似的。
- bad_chains0 的结果非常糟糕，这可以结合轨迹图得到佐证。
- bad_chains1 的结果相对较好，但其Rhat仍然大于1.01，说明该参数的收敛程度差，这点通过轨迹图有时难以发现其存在的问题。

Rhat的最低要求是低于1.1，但现在大多数发表的文章要求小于1.01。

#### 有效样本量 effective sample size (ESS) 与自相关 autocorrelation

前面通过自相关函数图发现，bad_chains1 会随着 n 间隔增大而自相关减少。

但这样做的代价是可用的样本数量变少。


![Image Name](https://cdn.kesci.com/upload/image/rkimhrvf7h.png?imageView2/0/w/640/h/640)


因此，我们是否能通过一个指标反应，在满足最小自相关的条件下，即只保留隔n个采样的样本时，可用的样本还剩下多少？

这就是 有效样本量(effective sample size,ESS)指标的意义。

ESS计算的逻辑为：
- 在样本量为n的 MCMC链中，分别计算 1到n 采样间隔下的样本相关性 $\rho$，这样可以计算得到，n个 $\rho_{n}$。
- 如果样本间自相关越大，$\rho$ 越接近1，相反则接近0。因此，n个 $\rho_{n}$ 相加，其值越接近n，说明自相关越大。
- 结合这个想法，用 n 比上 $\rho_{n}$ 的求和即可得到有效样本量。
  
  $ESS = \frac{N}{\sum_{t = -\infty}^{\infty} \rho_t}$

  如果自相关特别大，那么 ESS 接近1，如果自相关特别小，那么ESS接近n。

我们可以通过 `az.ess(.)` 函数计算 ESS。

In [80]:
az.ess(chains)

<xarray.Dataset>
Dimensions:      ()
Data variables:
    good_chains  float64 3.94e+03
    bad_chains0  float64 2.436
    bad_chains1  float64 89.97

可以看到，
- good_chains 的有效样本数量接近4000，说明样本的自相关问题少。
- bad_chains0 的有效样本量为2.4，该结果说明该 MCMC 样本的结果非常差。
- bad_chains1 的结果虽然比 bad_chains0 多，但是仍然很少。

一般建议 ESS 需要大于400，现在更多的要求是大于1000。

除了 ESS 外，另一个反应自相关的指标是 MCMC error，也叫 MCSE。

反映了 MCMC链的变异(方差)。其中，自相关越大，变异也越大。

我们可以通过 `az.mcse(.)` 函数计算 MCSE。

In [81]:
az.mcse(chains)

<xarray.Dataset>
Dimensions:      ()
Data variables:
    good_chains  float64 0.002507
    bad_chains0  float64 0.1067
    bad_chains1  float64 0.02095

结果显示：good_chains 的 MCSE最小，bad_chains0的结果最差。
这与 ESS 的解释相符。

In [82]:
az.summary(chains, kind="diagnostics")

,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
good_chains,0.003,0.002,3940.0,3883.0,1.00
bad_chains0,0.107,0.087,2.0,11.0,2.41
bad_chains1,0.021,0.017,90.0,182.0,1.03


### 如何改进 MCMC 采样过程

通过**视觉诊断**和**相应的指标**。我们了解了如何诊断 MCMC 样本可能出现的问题。

在诊断出 MCMC 样本可能存在的问题后，下一步就是如何修复这些问题。

常见的解决方法包括：
- 增加 MCMC 样本数量。
  
  比如将 `trace = pm.sample(draws = 2000)` 中的 draws 扩大为4000及其以上。

- 增加 MCMC链的数量。
  
  比如将 `trace = pm.sample(draws = 2000, chains=2)` 中的 chains 扩大为4。

- 设置burn-in。
  
  比如设置 `trace = pm.sample(draws = 2000, tune = 1000)` 这会采样3000个样本，丢掉最开的1000个样本。
	
	这样做的目的在于避免初始值太极端而导致花费太多迭代来达到平稳分布。
	![Image Name](https://cdn.kesci.com/upload/image/rkijy7xre5.png?imageView2/0/w/300/h/640)

这些方法的原理非常简单，目的都是增加有效样本数量。

但对于本身就不能收敛的模型，增加样本数量也是没有用的。

需要注意的，MCMC诊断只是 Baysian workflow 的其中一环。

当通过诊断不能发现问题，我们需要返回 workflow中通过其他方法去发现潜在的问题，

常见问题包括：
- 由于先验设置错误导致模型无法收敛。
- 变量单位不统一导致参数过大或过小。
- 数据量太少导致参数估计不准确。
- 模型设定错误，导致模型无法收敛。
- 编程代码错误，导致模型采样出现问题。

除了今天介绍的 MCMC 诊断方法 和 修复方法外，还有很多其他的方法，我们会再之后的实战中进行介绍

其他的诊断方法：
- Rank Plots 
![Image Name](https://cdn.kesci.com/upload/image/rkina0ih68.png?imageView2/0/w/320/h/320)
- Parallel plots
- Separation plots
- Divergences (HMC特有的问题及诊断方法)

source: https://arviz-devs.github.io/arviz/examples/index.html

其他修复方法：
- 设置 `target_accept`。
- 参见 workflow中的设置先验，参数重整化，模型结构调整等。